In [14]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [15]:
import os
import time
import pandas as pd

from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline
from netunicorn.library.tasks.capture.tcpdump import StartCapture, StopNamedCapture
from netunicorn.library.tasks.upload.fileio import UploadToFileIO
from netunicorn.library.tasks.upload.webdav import UploadToWebDav
from netunicorn.library.tasks.basic import SleepTask
from netunicorn.library.tasks.measurements.ookla_speedtest import SpeedTest
from netunicorn.library.tasks.video_watchers.youtube_watcher import WatchYouTubeVideo
from netunicorn.library.tasks.video_watchers.vimeo_watcher import WatchVimeoVideo
from netunicorn.library.tasks.video_watchers.twitch_watcher import WatchTwitchStream

In [16]:
NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'https://pinot.cs.ucsb.edu/netunicorn')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', 'cs190n16')       # substitute your login here
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', 'TEsjagDJ') # substitue your password here

In [17]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
print("Health Check: {}".format(client.healthcheck()))
nodes = client.get_nodes()
print(nodes)

Health Check: True
[[snl-server-5], <Uncountable node pool with next node template: [aws-fargate-A-cs190n16-, aws-fargate-B-cs190n16-, aws-fargate-ARM64-cs190n16-]>]


In [18]:
working_node = 'aws-fargate-A-cs190n16-'

In [19]:
pipeline = Pipeline()

# Generate data for YouTube
pipeline.then(StartCapture(filepath="/tmp/youtube_capture1.pcap", name="capture1"))
for _ in range(5):
    pipeline.then(WatchYouTubeVideo("https://www.youtube.com/watch?v=XxEhuSJF780", 10))
pipeline.then(StopNamedCapture(start_capture_task_name="capture1"))

# Upload Data
pipeline.then(UploadToWebDav(
    filepaths={"/tmp/youtube_capture1.pcap"}, 
    endpoint="http://localhost:8892/cs190n-fall-2024/project",  # Updated endpoint
    username="uploader",
    password="uploader"
))


Pipeline(2e3cf9b9-c9fe-4886-a0dc-538038440ba3): {'root': [<netunicorn.library.tasks.capture.tcpdump.StartCapture object at 0x7f3c73de6590>], 1: [<netunicorn.library.tasks.video_watchers.youtube_watcher.WatchYouTubeVideo object at 0x7f3c73de4760>], 2: [<netunicorn.library.tasks.video_watchers.youtube_watcher.WatchYouTubeVideo object at 0x7f3c73de4c40>], 3: [<netunicorn.library.tasks.video_watchers.youtube_watcher.WatchYouTubeVideo object at 0x7f3c73de5780>], 4: [<netunicorn.library.tasks.video_watchers.youtube_watcher.WatchYouTubeVideo object at 0x7f3c73de5480>], 5: [<netunicorn.library.tasks.video_watchers.youtube_watcher.WatchYouTubeVideo object at 0x7f3c73de7550>], 6: [<netunicorn.library.tasks.capture.tcpdump.StopNamedCapture object at 0x7f3c73de6cb0>], 7: [<netunicorn.library.tasks.upload.webdav.UploadToWebDav object at 0x7f3c73de6b00>]}

In [20]:
client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
print("Health Check: {}".format(client.healthcheck()))
nodes = client.get_nodes()
print(nodes)

working_nodes = nodes.filter(lambda node: node.name.startswith(working_node)).take(1)

# Creating the experiment
experiment = Experiment().map(pipeline, working_nodes)
print(experiment)

Health Check: True
[[snl-server-5], <Uncountable node pool with next node template: [aws-fargate-A-cs190n16-, aws-fargate-B-cs190n16-, aws-fargate-ARM64-cs190n16-]>]
 - Deployment: Node=aws-fargate-A-cs190n16-1, executor_id=, prepared=False, error=None


In [21]:
for line in experiment[0].environment_definition.commands:
    print(line)

sudo apt-get update
sudo apt-get install -y tcpdump
apt install -y python3-pip wget xvfb procps chromium chromium-driver
pip3 install selenium webdriver-manager
apt install -y python3-pip wget xvfb procps chromium chromium-driver
pip3 install selenium webdriver-manager
apt install -y python3-pip wget xvfb procps chromium chromium-driver
pip3 install selenium webdriver-manager
apt install -y python3-pip wget xvfb procps chromium chromium-driver
pip3 install selenium webdriver-manager
apt install -y python3-pip wget xvfb procps chromium chromium-driver
pip3 install selenium webdriver-manager
sudo apt-get update
sudo apt-get install -y tcpdump
sudo apt-get install -y procps
sudo apt-get install -y curl


In [22]:
from netunicorn.base import DockerImage
for deployment in experiment:
    # you can explore the image on the DockerHub
    deployment.environment_definition = DockerImage(image='speeeday/chromium-speedtest:0.3.1')
experiment_label = "datacoll"

In [23]:
try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

client.prepare_experiment(experiment, experiment_label)

while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(20)

ExperimentStatus.PREPARING
ExperimentStatus.READY


In [24]:
for deployment in client.get_experiment_status(experiment_label).experiment:
    print(f"Prepared: {deployment.prepared}, error: {deployment.error}")

Prepared: True, error: None


In [25]:
client.start_execution(experiment_label)

while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(20)

ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.FINISHED


In [26]:
from returns.pipeline import is_successful

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    result, log = report.result  # report stores results of execution and corresponding log
    
    # result is a returns.result.Result object, could be Success of Failure
    print(f"Result is: {type(result)}")
    data = result.unwrap() if is_successful(result) else result.failure()
    for key, value in data.items():
        print(f"{key}: {value}")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()

Node name: aws-fargate-A-cs190n16-1
Error: Executor timed out


TypeError: cannot unpack non-iterable NoneType object